In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms

# Training settings
b_size = 2048
n_epochs = 500
lr = 0.001
valid_step = 100
patience = max_patience = 10
best_loss = 100000
model_file = './models/model1.md'
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return F.log_softmax(out, dim=1)  

def test_validate(model, device, test_loader, test_valid='Test'):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\n{} set: Average loss: {:.6f}, Accuracy: {}/{} ({:.6f}%)\n'.format(
        test_valid, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss
    
def train(model, device, train_loader, valid_loader, optimizer, epoch):
    global best_loss, patience
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        if batch_idx % valid_step == 0:
            print('Start Validating...')
            valid_loss = test_validate(model, device, valid_loader, 'Valid')
            if valid_loss < best_loss:
                best_loss = valid_loss
                patience = max_patience
                print('Bese valid loss: {}'.format(best_loss))
                print('Improved! Reset patience.')
                print('Saving model...')
                torch.save(model, model_file)
            else:
                patience -= 1
                print('Not improved... Patience: {}'.format(patience))
                if patience == 0:
                    print('Out of patience. Stop training.')
                    return

def main():        
    dataroot = 'D:\Lab\dataset'

    device = torch.device("cuda")

    total_train_dataset = datasets.CIFAR10(dataroot, train=True, download=True,
                                           transform=transforms.Compose([
                                            transforms.RandomCrop(32, padding=4),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                       ]))
    total_train_size = len(total_train_dataset)
    split_ratio = 0.8
    train_size = int(split_ratio * total_train_size)
    valid_size = total_train_size - train_size
    
    train_dataset, valid_dataset = random_split(total_train_dataset,[train_size, valid_size] )
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=b_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=b_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(dataroot, train=False, transform=transforms.Compose([    
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                       ])),
        batch_size=b_size, shuffle=True)


    model = Net().to(device)
    print(model)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(1, n_epochs + 1):
        train(model, device, train_loader, valid_loader, optimizer, epoch)
        if patience == 0:
            break
                
    print('Start testing...')
    model = torch.load(model_file)
    test_validate(model, device, test_loader)
        
main()

Files already downloaded and verified
Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
Train Epoch: 1 [0/40000 (0%)]	Loss: 2.305157
Start Validating...

Valid set: Average loss: 2.302420, Accuracy: 1091/10000 (10.910000%)

Bese valid loss: 2.30241982421875
Improved! Reset patience.
Saving model...


C:\Users\Likit\Anaconda3\envs\tensorflow\lib\site-packages\torch\serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Train Epoch: 2 [0/40000 (0%)]	Loss: 2.129012
Start Validating...

Valid set: Average loss: 2.106884, Accuracy: 2290/10000 (22.900000%)

Bese valid loss: 2.106884423828125
Improved! Reset patience.
Saving model...
Train Epoch: 3 [0/40000 (0%)]	Loss: 1.973472
Start Validating...

Valid set: Average loss: 1.956725, Accuracy: 2833/10000 (28.330000%)

Bese valid loss: 1.9567249267578124
Improved! Reset patience.
Saving model...
Train Epoch: 4 [0/40000 (0%)]	Loss: 1.873665
Start Validating...

Valid set: Average loss: 1.849874, Accuracy: 3227/10000 (32.270000%)

Bese valid loss: 1.8498743896484375
Improved! Reset patience.
Saving model...
Train Epoch: 5 [0/40000 (0%)]	Loss: 1.786617
Start Validating...

Valid set: Average loss: 1.765841, Accuracy: 3522/10000 (35.220000%)

Bese valid loss: 1.7658408447265626
Improved! Reset patience.
Saving model...
Train Epoch: 6 [0/40000 (0%)]	Loss: 1.678885
Start Validating...

Valid set: Average loss: 1.711635, Accuracy: 3786/10000 (37.860000%)

Bese vali

Train Epoch: 42 [0/40000 (0%)]	Loss: 1.282407
Start Validating...

Valid set: Average loss: 1.307930, Accuracy: 5312/10000 (53.120000%)

Not improved... Patience: 8
Train Epoch: 43 [0/40000 (0%)]	Loss: 1.290932
Start Validating...

Valid set: Average loss: 1.303764, Accuracy: 5338/10000 (53.380000%)

Not improved... Patience: 7
Train Epoch: 44 [0/40000 (0%)]	Loss: 1.279872
Start Validating...

Valid set: Average loss: 1.286594, Accuracy: 5369/10000 (53.690000%)

Bese valid loss: 1.2865943359375
Improved! Reset patience.
Saving model...
Train Epoch: 45 [0/40000 (0%)]	Loss: 1.287396
Start Validating...

Valid set: Average loss: 1.286018, Accuracy: 5351/10000 (53.510000%)

Bese valid loss: 1.2860175537109375
Improved! Reset patience.
Saving model...
Train Epoch: 46 [0/40000 (0%)]	Loss: 1.295075
Start Validating...

Valid set: Average loss: 1.302157, Accuracy: 5360/10000 (53.600000%)

Not improved... Patience: 9
Train Epoch: 47 [0/40000 (0%)]	Loss: 1.272725
Start Validating...

Valid set: 


Valid set: Average loss: 1.158376, Accuracy: 5883/10000 (58.830000%)

Not improved... Patience: 8
Train Epoch: 87 [0/40000 (0%)]	Loss: 1.156045
Start Validating...

Valid set: Average loss: 1.157592, Accuracy: 5863/10000 (58.630000%)

Not improved... Patience: 7
Train Epoch: 88 [0/40000 (0%)]	Loss: 1.149302
Start Validating...

Valid set: Average loss: 1.151662, Accuracy: 5926/10000 (59.260000%)

Bese valid loss: 1.1516615600585938
Improved! Reset patience.
Saving model...
Train Epoch: 89 [0/40000 (0%)]	Loss: 1.145306
Start Validating...

Valid set: Average loss: 1.149765, Accuracy: 5930/10000 (59.300000%)

Bese valid loss: 1.149764501953125
Improved! Reset patience.
Saving model...
Train Epoch: 90 [0/40000 (0%)]	Loss: 1.152064
Start Validating...

Valid set: Average loss: 1.151390, Accuracy: 5872/10000 (58.720000%)

Not improved... Patience: 9
Train Epoch: 91 [0/40000 (0%)]	Loss: 1.125235
Start Validating...

Valid set: Average loss: 1.144579, Accuracy: 5923/10000 (59.230000%)

Bese 